# Defining the Model

In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CarDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        drag_coefficient = torch.tensor(float(self.annotations.iloc[idx, 1]))

        if self.transform:
            image = self.transform(image)

        return image, drag_coefficient

# Step 2: Define a Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 3: Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Step 4: Split the Dataset
# Assume you have already split your dataset into train and validation sets

# Step 5: Define Loss Function
criterion = nn.MSELoss()

# Step 6: Choose an Optimizer
model = CNNModel()
optimizer = optim.Adam(model.parameters(), lr=0.002)

# Training the Model

In [4]:
# Step 4: Split the Dataset
# Since you only have a training set, you can skip this step

# Step 7: Training Loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=15):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Step 8: Evaluation (Optional)
# If you don't have a separate validation set, you can skip the evaluation step

# Step 11: Putting It All Together
if __name__ == "__main__":
    train_dataset = CarDataset(csv_file='norm dataset/annotations.csv', root_dir='norm dataset/edgelefts JPG', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    train_model(model, train_loader, criterion, optimizer)




Epoch [1/15], Loss: 0.0050
Epoch [2/15], Loss: 0.0050
Epoch [3/15], Loss: 0.0050
Epoch [4/15], Loss: 0.0050
Epoch [5/15], Loss: 0.0050
Epoch [6/15], Loss: 0.0050
Epoch [7/15], Loss: 0.0050
Epoch [8/15], Loss: 0.0050
Epoch [9/15], Loss: 0.0050
Epoch [10/15], Loss: 0.0050
Epoch [11/15], Loss: 0.0050
Epoch [12/15], Loss: 0.0050
Epoch [13/15], Loss: 0.0050
Epoch [14/15], Loss: 0.0050
Epoch [15/15], Loss: 0.0050


In [5]:
torch.save(model.state_dict(), 'engine.pth')

# Loading in the model

In [6]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Define your model architecture (the same as the one used during training)
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CNNModel()

# Load the model's state dictionary
model.load_state_dict(torch.load('engine.pth'))

# Set the model to evaluation mode
model.eval()


CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

# Testing with an image

In [10]:
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Define a function to predict the drag coefficient
def predict_drag_coefficient(image_path, model, transform):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)  # Add a batch dimension
    with torch.no_grad():
        output = model(image_tensor)
    predicted_coefficient = output.item()
    return predicted_coefficient

# Input a car outline image and get the predicted drag coefficient
image_path = 'testing_normal/003_INVICTO_P_side-1200x801.jpg'
predicted_coefficient = predict_drag_coefficient(image_path, model, test_transform)
print(f'Predicted drag coefficient: {predicted_coefficient}')

Predicted drag coefficient: 0.43892672657966614
